In [ ]:
from IPython.display import HTML

HTML("""
<style>
/* JupyterLab: center notebook and leave margins */
.jp-NotebookPanel-notebook {
  width: 85% !important;              
  margin-left: auto !important;
  margin-right: auto !important;
  max-width: 1100px !important;       /* optional cap */
}

/* Make output area match nicely */
.jp-OutputArea {
  max-width: 100% !important;
}
</style>
""")

%config InlineBackend.figure_format = 'svg'

## 0. Preparing Your Environment

### 0.1 Install Anaconda and your environment

**If you are using Windows, I strongly recommend installing Ubuntu 22.04 via [WSL](https://ubuntu.com/desktop/wsl).** If you are using macOS or Linux, you are ready to go.


- **Step 1: Install Anaconda.** Download and install Anaconda from: [https://www.anaconda.com/download](https://www.anaconda.com/download).

- **Step 2: Create and activate the course environment.** Install your conda environment

  - If you have GPU in your machine (Linux + NVIDIA GPU), please create your env via

    ```conda env create -f environment-gpu.yml```

    Activate it with:

    ```conda activate llm-26-gpu```

    Note: environment-gpu.yml uses ```pytorch-cuda=12.1```. If your GPU driver/CUDA setup does not support CUDA 12.1, change this version accordingly (e.g., 11.8).

  - If you use macOS / Windows (WSL) / Linux CPU-only, please create your env via
 
    ```conda env create -f environment.yml```
 
    Activate it with:
 
    ```conda activate llm-26```

- Note, if you got errors when installing ```en_core_web_sm``` or ```zh_core_web_sm```, please remove these two from yml file and install separately in your env via

  ```shell
  conda activate llm-26 # or conda activate llm-26-gpu
  python -m spacy download en_core_web_sm
  python -m spacy download zh_core_web_sm
  ```
  After the above steps, you are ready to download our course github project. Make sure you have [git](https://git-scm.com/install/) in your system.

  ```git clone git@github.com:baojian/llm-26.git```

  After the above steps, you are ready to open our jupyter notebook.

- **Step 3: Open your jupyter notebook.**

  All your code will run on Jupyter notebook, you can activate jupyter notebook, via

  ```conda activate llm-26 # or conda activate llm-26-gpu```
  
  ```jupyter lab```

  For students using Windows WSL (Ubuntu 22.04), even though Jupyter runs inside WSL (Ubuntu), you can open it directly in the Windows browser via port forwarding (usually automatic). In WSL Ubuntu, start Jupyter like this:

  ```jupyter lab --no-browser --ip=127.0.0.1 --port=8888```

  It will print a URL like: ```http://127.0.0.1:8888/lab?token=...```

  Now on Windows, open your browser and go to: ```http://localhost:8888/lab``` Paste the token if it asks.

  ✅ On WSL2, Windows automatically forwards localhost:8888 to the WSL instance in most setups. If localhost:8888 doesn’t work, in WSL run: ```hostname -I```, suppose it prints something like 172.27.123.45 ..., then open in Windows: ```http://172.27.123.45:8888/lab```

### 0.2 Checking your device

After install conda and your env, you can check whether these packages are installed in the right way.
```shell
python -c "import torch; print('torch', torch.__version__); print('cuda?', torch.cuda.is_available()); print('mps?', getattr(torch.backends,'mps',None) is not None and torch.backends.mps.is_available()); print('cuda device:', torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'no gpu'); print('using:', 'cuda' if torch.cuda.is_available() else ('mps' if (getattr(torch.backends,'mps',None) is not None and torch.backends.mps.is_available()) else 'cpu'))"
```

In [30]:
!python -c "import torch; print('torch', torch.__version__); print('cuda?', torch.cuda.is_available()); print('mps?', getattr(torch.backends,'mps',None) is not None and torch.backends.mps.is_available()); print('cuda device:', torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'no gpu'); print('using:', 'cuda' if torch.cuda.is_available() else ('mps' if (getattr(torch.backends,'mps',None) is not None and torch.backends.mps.is_available()) else 'cpu'))"

torch 2.9.1
cuda? False
mps? True
cuda device: no gpu
using: mps


- The following code provides a more symmetric way to perform the same checks.

In [31]:
import torch

def detect_torch_device(verbose: bool = True) -> str:
    """
    Returns one of: 'cuda', 'mps', 'cpu'
    Priority: CUDA GPU > Apple MPS > CPU
    """
    has_cuda = torch.cuda.is_available()
    has_mps = getattr(torch.backends, "mps", None) is not None and torch.backends.mps.is_available()

    if has_cuda:
        device = "cuda"
    elif has_mps:
        device = "mps"
    else:
        device = "cpu"

    if verbose:
        print(f"torch: {torch.__version__}")
        print(f"device: {device}")

        if has_cuda:
            print(f"cuda devices: {torch.cuda.device_count()}")
            for i in range(torch.cuda.device_count()):
                print(f"  [{i}] {torch.cuda.get_device_name(i)}")
        elif has_mps:
            print("mps available: True (Apple Metal)")
        else:
            print("cpu only")

    return device

device = detect_torch_device()
# generate 2x3 random matrix to check torch
x = torch.rand(2, 3)
x = x.to(device)
print("device:", x.device)

torch: 2.9.1
device: mps
mps available: True (Apple Metal)
device: mps:0


During our lectures, some datasets are very large. To make sure you have enough disk space, you can check your disk, memory and cpu information via the following code.

In [32]:
import os
import platform
import shutil

def bytes_to_gb(x: int) -> float:
    return x / (1024 ** 3)

def system_report(path: str = "."):
    print("=== System Report ===")
    print("OS:", platform.system(), platform.release())
    print("Platform:", platform.platform())
    print("Python:", platform.python_version())

    # CPU
    print("\n--- CPU ---")
    print("CPU cores (logical):", os.cpu_count())

    # Memory (best-effort, cross-platform)
    print("\n--- Memory (RAM) ---")
    try:
        import psutil  # you already have this in env
        vm = psutil.virtual_memory()
        print(f"Total: {bytes_to_gb(vm.total):.2f} GB")
        print(f"Available: {bytes_to_gb(vm.available):.2f} GB")
        print(f"Used: {bytes_to_gb(vm.used):.2f} GB ({vm.percent}%)")
    except Exception as e:
        print("psutil not available or failed:", e)

    # Disk
    print("\n--- Disk ---")
    total, used, free = shutil.disk_usage(path)
    print("Path checked:", os.path.abspath(path))
    print(f"Total: {bytes_to_gb(total):.2f} GB")
    print(f"Free:  {bytes_to_gb(free):.2f} GB")
    print(f"Used:  {bytes_to_gb(used):.2f} GB")

    # PyTorch device
    print("\n--- PyTorch Device ---")
    try:
        import torch
        cuda = torch.cuda.is_available()
        mps = hasattr(torch.backends, "mps") and torch.backends.mps.is_available()
        print("torch:", torch.__version__)
        print("CUDA available:", cuda)
        print("MPS available:", mps)
        if cuda:
            print("GPU:", torch.cuda.get_device_name(0))
        device = "cuda" if cuda else ("mps" if mps else "cpu")
        print("Suggested device:", device)
    except Exception as e:
        print("torch not available or failed:", e)

system_report(".")

=== System Report ===
OS: Darwin 24.3.0
Platform: macOS-15.3.1-arm64-arm-64bit
Python: 3.12.9

--- CPU ---
CPU cores (logical): 12

--- Memory (RAM) ---
Total: 18.00 GB
Available: 2.88 GB
Used: 7.29 GB (84.0%)

--- Disk ---
Path checked: /Users/baojianzhou/git/llm-26/lecture-01
Total: 926.35 GB
Free:  320.82 GB
Used:  605.53 GB

--- PyTorch Device ---
torch: 2.9.1
CUDA available: False
MPS available: True
Suggested device: mps


## 1. Explore LLMs via ollama

### 1.1  Install ollama and download Qwen3:0.6b and Qwen3:1.7b

We can download some popular LLMs such as Qwen series.

- Download ollama to your laptop from [https://ollama.com/download](https://ollama.com/download)
- Run ```shell ollama run qwen3:0.6b ```, it will automatically download the model into your local disk. It takes about 498MB disk space.
- Similarly, you can run, ```shell ollama run qwen3:1.7b ```. It takes about 1.3GB disk space.

You can install the python api of ollama via the following commands.

In [ ]:
!conda install conda-forge::ollama -y

In [ ]:
!conda install -c conda-forge ollama-python -y

### 1.2 Get response for a given prompt

In [ ]:
import os
import time
import math
import ollama

# If you have proxy, make sure bypass proxies for local services (e.g., Ollama on localhost:11434)
# You can check whether Ollama is running or not by tpying: http://127.0.0.1:11434/ in your Chrome.
os.environ["NO_PROXY"] = "localhost,127.0.0.1"
os.environ["no_proxy"] = "localhost,127.0.0.1"

models = ["qwen3:0.6b", "qwen3:1.7b"]
prompt = "Fudan University is located in which city? Answer with one word."

for model in models:
    print('-' * 50)
    start_time = time.time()
    for _ in range(10):
        resp = ollama.generate(
            model = model,
            prompt = prompt
        )
        print(f"{model} with resp {_ + 1}: {resp["response"]}")
    print(f'total runtime of 10 responses of {model} is: {time.time() - start_time:.1f} seconds')

**Some key observations:**

- The smaller model Qwen3:0.6b produces lower quality answers while the relative bigger model Qwen3:1.7b produces high quality answers.
- The response is a kind of random as each time the response may not be the same.

Certainly, there are ways to make sure the above to generate a fixed answer. For example, you can use the following code where each time it always produce the maximal probability as the answer:

```python
resp = ollama.generate(
    model=model,
    prompt=prompt,
    options={
        "temperature": 0.0,
        "top_p": 1.0,
        "top_k": 0,
        # optional:
        "num_predict": 32,
    },
)
print(resp["response"])
```

Let us generate some response that are not one word but a sequence of words.

In [ ]:
model = "qwen3:1.7b"
prompt = "I am an undergraduate student, please explain LLMs in three sentences."
resp = ollama.generate(
            model=model,
            prompt=prompt
        )
print(f"Prompt: {prompt} \nResp: {resp["response"]}")

### 1.3 Get response probability from LLMs

From above outputs, you see that each time, the response of these LLMs could be different. Actually, all LLMs are probabilitic model where each time, they response (i.e., answers) prompts (i.e., questions) differently. We can use math language to describe this inference process.

Let $p_\theta(\cdot)$ be the trained probability model. Here, you can think $p_\theta(\cdot)$ as Qwen3:0.6b, Qwen3:1.7b or any other models. Given the prompt *Fudan University is located in which city? Answer with one word.* (a sequence of tokens), the above response will give you an answer, which is also a sequence of tokens. So, the model will use an algorithm to predict the response given the prompt. Use the math language, we can think it tries to calculate the following probability:

$$
p_\theta \left(w_{n+1}|w_1,w_2,\ldots, w_{n}\right),
$$
where
- Prompt=[*Fudan University is located in which city? Answer with one word.]* $=[w_1,w_2,\ldots, w_{n}]$,
- Response=[$w_{n+1}$].

The model will try to output the most likeliy word $w_{n+1}$ using its own *inference algorithm*. We will detail this part in later lectures. By the definition of conditional probability, we may calculate it as

$$
p_\theta \left(w_{n+1}|w_1,w_2,\ldots, w_{n}\right) = \frac{ p_\theta \left(w_1,w_2,\ldots, w_{n}, w_{n+1}\right)}{p_\theta \left(w_1,w_2,\ldots, w_{n}\right)}.
$$

The above is essential to say that, if we can learn a model that can represent any length sequence of distribution $p_\theta \left(w_1,w_2,\ldots, w_{k}\right)$, where $k$ could be any positive integer, then we can compute the conditional distribution easily.

In [ ]:
model = "qwen3:0.6b" # "qwen3:1.7b"
prompt = "Fudan University is located in which city? Answer with one word."
num_top_tokens = 20 # number of top alternatives per generated token
resp = ollama.generate(
    model = model,
    prompt = prompt,
    stream = False,
    logprobs = True,
    think = False,
    top_logprobs = num_top_tokens
)
print("response:", repr(resp["response"]))

# Each element usually corresponds to one generated token
for i, lp in enumerate(resp.get("logprobs", [])):
    tok = lp.get("token")
    logp = lp.get("logprob")
    p = math.exp(logp) if logp is not None else None
    print(f"{i:02d} token={tok!r:>16} logp={logp: .4f} p={p:.4f}")

In [ ]:
import os, math, ollama

model = "qwen3:0.6b"
prompt = "Fudan University is located in which city? Answer with one word."

res = ollama.generate(
    model=model,
    prompt=prompt,
    logprobs=True,
    think = False,
    top_logprobs=10,
    options={"temperature": 0.0, # greedy decoding, it pick the maximal token
             "num_predict": 20,
            "think": False # do not use thinking model.
            },
)

answer = ''
lp = res["logprobs"]
tokens = [d.get("token", "") for d in lp]
print(f'We use model: {model}')
for i in range(len(lp)):
    tok = lp[i].get("token", "")
    logp = lp[i].get("logprob", None)
    alts = lp[i].get("top_logprobs", [])
    p = math.exp(logp) if logp is not None else float("nan")
    if tok == "\n" or tok == "\n\n": # stop when answer ends (often newline).
        break
    answer += tok
    print(f"--- top probabilities of token-{i:02d} ---")
    for a in alts[:20]:
        prob_a = math.exp(a['logprob'])
        print(f"{a['token']!r:>12}:{prob_a:.5f}")
    print(f"Partial Response: {answer}\n")
print(f"Final Response: {answer}")

In [ ]:
model = "qwen3:1.7b"
prompt = "Fudan University is located in which city? Answer with one word."

res = ollama.generate(
    model=model,
    prompt=prompt,
    logprobs=True,
    think = False,
    top_logprobs=10,
    options={"temperature": 0.0, # greedy decoding, it pick the maximal token
             "num_predict": 20,
            "think": False # do not use thinking model.
            },
)
print(res["response"])

If we do not use thinking mode, it will gives the above response. However, we can still see how Shanghai is chosen during the inference stage.

In [ ]:
import os, math, ollama

model = "qwen3:1.7b"
prompt = "Fudan University is located in which city? Answer with one word."

res = ollama.generate(
    model=model,
    prompt=prompt,
    logprobs=True,
    think = False, # Do not use the thinking model.
    top_logprobs=10,
    options={"temperature": 0.0, # greedy decoding, it pick the maximal token
             "num_predict": 20,
            "think": False # do not use thinking model.
            },
)

answer = ''
lp = res["logprobs"]
tokens = [d.get("token", "") for d in lp]
print(f'We use model: {model}')
for i in range(len(lp)):
    tok = lp[i].get("token", "")
    logp = lp[i].get("logprob", None)
    alts = lp[i].get("top_logprobs", [])
    p = math.exp(logp) if logp is not None else float("nan")
    if tok == "\n" or tok == "\n\n": # stop when answer ends (often newline).
        break
    answer += tok
    print(f"--- top probabilities of token-{i:02d} ---")
    for a in alts[:20]:
        prob_a = math.exp(a['logprob'])
        print(f"{a['token']!r:>12}:{prob_a:.5f}")
    print(f"Partial Response: {answer}\n")
print(f"Final Response: {answer}")

## 2. Datasets Exploration

Please check this part in our slides.

### 2.1 Explore wikitext-2 dataset

Please note that huggingface cannot be directly used in China. An alternative way is to use [https://hf-mirror.com/](https://hf-mirror.com/).

In [ ]:
import os
# If this does not work, please add export HF_ENDPOINT=https://hf-mirror.com in your env.
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
os.environ["HF_HUB_ETAG_TIMEOUT"] = "60"
os.environ["HF_HUB_DOWNLOAD_TIMEOUT"] = "60"

from datasets import load_dataset

# loads train/validation/test
ds = load_dataset("wikitext", "wikitext-2-raw-v1")

In [ ]:
print(ds)
train = ds["train"]
for i in range(4):
    print(train[i])

In [ ]:
import re
import numpy as np
import matplotlib.pyplot as plt

train = ds["train"]

# simple word tokenizer (lowercased)
word_re = re.compile(r"[A-Za-z]+(?:'[A-Za-z]+)?")

def heaps_curve(dataset, step=1000):
    V = set()
    N = 0
    Ns, Vs = [], []

    for ex in dataset:
        text = ex["text"]
        if not text or text.strip() == "": # empty word -> continue
            continue
        # make all words to low cases
        words = word_re.findall(text.lower()) 
        for w in words:
            N += 1
            V.add(w)

            if N % step == 0:
                Ns.append(N)
                Vs.append(len(V))
    return np.array(Ns), np.array(Vs)

Ns, Vs = heaps_curve(train, step=1000)

# Fit log |V| = log k + beta log N  -> linear regression on logs
logN = np.log(Ns)
logV = np.log(Vs)
beta, logk = np.polyfit(logN, logV, 1)
k = np.exp(logk)

print(f"Fitted Heaps' law: |V| ≈ {k:.2f} * N^{beta:.3f}")

# Plot (log-log)
plt.figure(figsize=(7, 5))
plt.loglog(Ns, Vs, marker='o', linestyle='none', markersize=5, label="Empirical (Wikitext-2)")

# fitted line
Ns_line = np.linspace(Ns[0], Ns[-1], 200)
Vs_line = k * (Ns_line ** beta)
plt.loglog(Ns_line, Vs_line, label=fr"Fitted ($k={k:.3f},\beta=${beta:.3f})")

plt.xlabel("$N$ (tokens)", fontsize = 15)
plt.ylabel("$|V|$ (vocab)", fontsize = 15)
plt.title(r"Heaps' Law (Simple tokenization): $|V|=kN^{\beta}$", fontsize = 15)
plt.legend(fontsize = 15)
plt.tight_layout()
plt.show()

Next, we can use spacy to do the tokenization.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import spacy

nlp = spacy.load("en_core_web_sm", disable=["tagger","parser","ner","lemmatizer"])
# tokenizer still works even with pipeline disabled

train = ds["train"]

def heaps_curve_spacy(dataset, step=10_000, batch_size=256):
    V = set()
    N = 0
    Ns, Vs = [], []

    texts = (ex["text"] for ex in dataset if ex["text"] and ex["text"].strip())
    for doc in nlp.pipe(texts, batch_size=batch_size):
        for tok in doc:
            # choose your definition of "word"
            if tok.is_alpha:
                w = tok.text.lower()
                N += 1
                V.add(w)
                if N % step == 0:
                    Ns.append(N)
                    Vs.append(len(V))
    return np.array(Ns), np.array(Vs)

Ns, Vs = heaps_curve_spacy(train, step=10_000)

# Fit log |V| = log k + beta log N
logN = np.log(Ns)
logV = np.log(Vs)
beta, logk = np.polyfit(logN, logV, 1)
k = np.exp(logk)

print(f"Fitted Heaps' law (spaCy): |V| ≈ {k:.2f} * N^{beta:.3f}")
# Plot (log-log)
plt.figure(figsize=(7, 5))
plt.loglog(Ns, Vs, marker='o', linestyle='none', markersize=5, label="Empirical (Wikitext-2)")

# fitted line
Ns_line = np.linspace(Ns[0], Ns[-1], 200)
Vs_line = k * (Ns_line ** beta)
plt.loglog(Ns_line, Vs_line, label=fr"Fitted ($k={k:.3f},\beta=${beta:.3f})")

plt.xlabel("$N$ (tokens)", fontsize = 15)
plt.ylabel("$|V|$ (vocab)", fontsize = 15)
plt.title(r"Heaps' Law (Spacy tokenization): $|V|=kN^{\beta}$", fontsize = 15)
plt.legend(fontsize = 15)
plt.tight_layout()
plt.show()

### 2.2 Explore wikitext-103 dataset

We can download a larger dataset from the following:

```shell
export HF_HOME=$HOME/.cache/huggingface

huggingface-cli download Salesforce/wikitext --repo-type dataset --resume-download --include "wikitext-103-raw-v1/*.parquet"
```

In [ ]:
from datasets import load_dataset
ds = load_dataset("wikitext", "wikitext-103-raw-v1")  # will hit cache if present

In [ ]:
print(ds)
train = ds["train"]
for i in range(4):
    print(train[i])

In [ ]:
train = ds["train"]
# simple word tokenizer (lowercased)
word_re = re.compile(r"[A-Za-z]+(?:'[A-Za-z]+)?")

def heaps_curve(dataset, step=1000):
    V = set()
    N = 0
    Ns, Vs = [], []

    for ex in dataset:
        text = ex["text"]
        if not text or text.strip() == "": # empty word -> continue
            continue
        # make all words to low cases
        words = word_re.findall(text.lower()) 
        for w in words:
            N += 1
            V.add(w)

            if N % step == 0:
                Ns.append(N)
                Vs.append(len(V))
    return np.array(Ns), np.array(Vs)

Ns, Vs = heaps_curve(train, step=1000)

# Fit log |V| = log k + beta log N  -> linear regression on logs
logN = np.log(Ns)
logV = np.log(Vs)
beta, logk = np.polyfit(logN, logV, 1)
k = np.exp(logk)

print(f"Fitted Heaps' law (spaCy): |V| ≈ {k:.2f} * N^{beta:.3f}")
# Plot (log-log)
plt.figure(figsize=(7, 5))
plt.loglog(Ns, Vs, marker='o', linestyle='none', markersize=5, label="Empirical (Wikitext-103)")

# fitted line
Ns_line = np.linspace(Ns[0], Ns[-1], 200)
Vs_line = k * (Ns_line ** beta)
plt.loglog(Ns_line, Vs_line, label=fr"Fitted ($k={k:.3f},\beta=${beta:.3f})")

plt.xlabel("$N$ (tokens)", fontsize = 15)
plt.ylabel("$|V|$ (vocab)", fontsize = 15)
plt.title(r"Heaps' Law (Spacy tokenization): $|V|=kN^{\beta}$", fontsize = 15)
plt.legend(fontsize = 15)
plt.tight_layout()
plt.show()

### 2.3 BookCorpus datasets

BookCorpus dataset has been used in training GPT-1. See <a href="/files/papers/GPT1-Improving_Language_Understanding_by_Generative_Pre-Training.pdf"
   target="_blank" rel="noopener">
  GPT-1 paper (PDF)
</a>

In [ ]:
import time
import tarfile

path = "../.datasets/books1.tar.gz"
start_time = time.time()
with tarfile.open(path, "r:gz") as tar:
    names = tar.getnames()
    print("num members:", len(names))
    print("\n".join(names[:50]))
print(f"total time to scan BookCorpus: {time.time() - start_time:.2f} seconds")

You can check ```shell tokenization_bookcorpus.py``` to generate heaps statistics in jsonl file.

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

def load_heaps_jsonl(log_path: str):
    Ns, Vs = [], []
    with open(log_path, "r", encoding="utf-8") as f:
        for line in f:
            r = json.loads(line)
            Ns.append(r["total_tokens"])
            Vs.append(r["vocab_size"])
    return np.array(Ns), np.array(Vs)

Ns, Vs = load_heaps_jsonl("books1_wordcounts.heaps_log.jsonl")

# fit log |V| = log k + beta log N
beta, logk = np.polyfit(np.log(Ns), np.log(Vs), 1)
k = np.exp(logk)
print(f"Fit: |V| ≈ {k:.2f} * N^{beta:.3f}")


plt.figure(figsize=(7,5))

plt.loglog(Ns, Vs, marker="o", linestyle="none", 
           markersize=5, label="Empirical (BookCorpus)")
Ns_line = np.linspace(Ns[0], Ns[-1], 200)
plt.loglog(Ns_line, k * (Ns_line ** beta), label=fr"Fitted Curve $(k={k:.3f},\beta=${beta:.3f})")
plt.xlabel("$N$ (tokens)", fontsize = 14)
plt.ylabel("$|V|$ (vocab)", fontsize = 14)
plt.title(r"Heaps' Law (Simple tokenization): $|V|=k N^{\beta}$", fontsize = 14)
plt.legend(fontsize = 14)
plt.tight_layout()
plt.show()

## 3. Basics for Python and spaCy

- **Python**: We will use Python-3.12 in our course.
- **spaCy**: As introduced in [https://github.com/explosion/spaCy](https://github.com/explosion/spaCy), [spaCy](https://spacy.io/) is a library for advanced Natural Language Processing in Python and Cython. It's built on the very latest research, and was designed from day one to be used in real products. We will use it to demonostrate how to do text tokenization.
- **nltk tool**: In our previous courses, we will introduce nltk tool for tokenization stuff. But we will not cover this part in our new course as these tools are largely iirelvant and outdated. One can find details of nltk at [https://github.com/nltk/nltk](https://github.com/nltk/nltk) and website at [https://www.nltk.org/](https://www.nltk.org/).

### 3.1 Python basics

In [ ]:
# in Python, there is a built in lib re, we can import them
import re


In [ ]:
# Task: Find woodchuck or Woodchuck : Disjunction
test_str = "This string contains Woodchuck and woodchuck."
result=re.search(pattern="[wW]oodchuck", string=test_str)
print(result)
result=re.search(pattern=r"[wW]ooodchuck", string=test_str)
print(result)

In [ ]:
# Find the word "woodchuck" in the following test string
test_str = "interesting links to woodchucks ! and lemurs!"
re.search(pattern="woodchuck", string=test_str)

In [ ]:
# Find !, it follows the same way:
print(re.search(pattern="!", string=test_str))
print(re.search(pattern="!!", string=test_str))
assert re.search(pattern="!!", string=test_str) == None # match nothing

In [ ]:
# Find any single digit in a string.
result=re.search(pattern=r"[0123456789]", string="plenty of 7 to 5")
print(result)
result=re.search(pattern=r"[0-9]", string="plenty of 7 to 5")
print(result)

In [ ]:
# Negation: If the caret ^ is the first symbol after [,
# the resulting pattern is negated. For example, the pattern 
# [^a] matches any single character (including special characters) except a.

# -- not an upper case letter
print(re.search(pattern=r"[^A-Z]", string="Oyfn pripetchik"))

# -- neither 'S' nor 's'
print(re.search(pattern=r"[^Ss]", string="I have no exquisite reason for't"))

# -- not a period
print(re.search(pattern=r"[^.]", string="our resident Djinn"))

# -- either 'e' or '^'
print(re.search(pattern=r"[e^]", string="look up ^ now"))

# -- the pattern ‘a^b’
print(re.search(pattern=r'a\^b', string=r'look up a^b now'))

In [ ]:
# More disjuncations
str1 = "Woodchucks is another name for groundhog!"
result = re.search(pattern="groundhog|woodchuck",string=str1)
print(result)

In [ ]:
str1 = "Find all woodchuckk Woodchuck Groundhog groundhogxxx!"
result = re.findall(pattern="[gG]roundhog|[Ww]oodchuck",string=str1)
print(result)

In [ ]:
# Some special chars

# ?: Optional previous char
str1 = "Find all color colour colouur colouuur colouyr"
result = re.findall(pattern="colou?r",string=str1)
print(result)

# *: 0 or more of previous char
str1 = "Find all color colour colouur colouuur colouyr"
result = re.findall(pattern="colou*r",string=str1)
print(result)

# +: 1 or more of previous char
str1 = "baa baaa baaaa baaaaa"
result = re.findall(pattern="baa+",string=str1)
print(result)
# .: any char
str1 = "begin begun begun beg3n"
result = re.findall(pattern="beg.n",string=str1)
print(result)
str1 = "The end."
result = re.findall(pattern="\.$",string=str1)
print(result)
str1 = "The end? The end. #t"
result = re.findall(pattern=".$",string=str1)
print(result)

In [ ]:
# find all "the" in a raw text.
text = "If two sequences in an alignment share a common ancestor, \
mismatches can be interpreted as point mutations and gaps as indels (that \
is, insertion or deletion mutations) introduced in one or both lineages in \
the time since they diverged from one another. In sequence alignments of \
proteins, the degree of similarity between amino acids occupying a \
particular position in the sequence can be interpreted as a rough \
measure of how conserved a particular region or sequence motif is \
among lineages. The absence of substitutions, or the presence of \
only very conservative substitutions (that is, the substitution of \
amino acids whose side chains have similar biochemical properties) in \
a particular region of the sequence, suggest [3] that this region has \
structural or functional importance. Although DNA and RNA nucleotide bases \
are more similar to each other than are amino acids, the conservation of \
base pairs can indicate a similar functional or structural role."
matches = re.findall("[^a-zA-Z][tT]he[^a-zA-Z]", text)
print(matches)

In [ ]:
# A nicer way is to do the following

matches = re.findall(r"\b[tT]he\b", text)
print(matches)

In [ ]:
def get_wikilink_re():
    """ This regex is from the following Github:
    https://github.com/WikiLinkGraphs/wikidump
    """
    regex_str = r'''(?P<total>(?P<wikilink>
        \[\[(?P<link>[ˆ\n\|\]\[\<\>\{\}]{0,256})(?:\|(?P<anchor>[ˆ\[]*?))?\]\])\w*)\s?'''
    return regex.compile(regex_str, regex.VERBOSE | regex.MULTILINE)

# Task: Implement the task shown in Slides 52
# You may need to
# 1. Download a Wikipedia article xml file
# 2. Use RE to extract links.

### 3.2 spaCy

In [ ]:
## install spaCy
!conda install -c conda-forge spacy -y --quiet

Open your terminal and activate your llm-26 env, and then run the following to download English LMs.
```python
python -m spacy download en_core_web_sm
```

In [ ]:
import spacy

prompt = "Fudan University is located in which city? Answer with one word."
nlp = spacy.load("en_core_web_sm")
doc = nlp(prompt) # i want to do text preprocessing for our prompt.

# Text: The original word text.
# Lemma: The base form of the word.
# POS: The simple UPOS part-of-speech tag.
# Tag: The detailed part-of-speech tag.
# Dep: Syntactic dependency, i.e. the relation between tokens.
# Shape: The word shape – capitalization, punctuation, digits.
# is alpha: Is the token an alpha character? (whether it consists only of letters from the alphabet (A-Z or a-z))
# is stop: Is the token part of a stop list, i.e. the most common words of the language? 
#         (A stop list (or stopwords list) is a list of commonly used words in a language that 
#         are usually ignored during natural language processing (NLP) tasks, such as text analysis or machine learning.)

for token in doc:
    print(f"--- token: {token.text} ---")
    print(f"lemma: {token.lemma_}\npos: {token.pos_}\ntag: {token.tag_}\ndep: {token.dep_}\nshape: {token.shape_}\nis_alpha:{token.is_alpha}\nis_stop: {token.is_stop}")

There are two type of tokenizations

- **Top-down tokenization**: We define a standard and implement rules to implement that kind of tokenization.
  - word tokenization
  - charater tokenization
- **Bottom-up tokenization**: We use simple statistics of letter sequences to break up words into subword tokens.
  - subword tokenization (modern LLMs use this type!)

In [ ]:
# Use split method via the whitespace " "
text = """While the Unix command sequence just removed all the numbers and punctuation"""
print(text.split(" "))

In [ ]:
# But, we have punctuations, icons, and many other small issues.
text = """Don't you love 🤗 Transformers? We sure do."""
print(text.split(" "))

In [ ]:
# spacy works much better
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(text)
print([token for token in doc])

In [ ]:
text = """Special characters and numbers will need to be kept in prices ($45.55) and dates (01/02/06); 
we don’t want to segment that price into separate tokens of “45” and “55”. And there are URLs (https://www.stanford.edu),
Twitter hashtags (#nlproc), or email addresses (someone@cs.colorado.edu)."""
doc = nlp(text)
print([token for token in doc])

Please install zh tokenization via spaCy: 
```python
python -m spacy download zh_core_web_sm
```

In [ ]:
nlp = spacy.load("zh_core_web_sm")
text = '姚明进入总决赛'
doc = nlp(text)
print([token for token in doc])

In [ ]:
text = """1月1日，国务院国资委公布“2025年度央企十大国之重器”，1月2日公布“2025年度央企十大超级工程”。‌‌"""
doc = nlp(text)
print([str(token) for token in doc])

In [ ]:
from spacy.lang.zh import Chinese
nlp_ch = Chinese()
print([*nlp_ch(text)])

**Lemmatization (词形还原)**

- Lemmatization is the task of determining that two words have the same root, despite their surface differences.
- **Motivation**: For some NLP situations, we also want two morphologically different forms of a word to behave similarly. For example in web search, someone may type the string woodchucks but a useful system might want to also return pages
that mention woodchuck with no s.
- **Example 1**: The words am, are, and is have the shared lemma be.
- **Example 2**: The words dinner and dinners both have the lemma dinner.

In [ ]:
text = """
The Brown Corpus, a text corpus of American English that was compiled in the 1960s at Brown University, \
is widely used in the field of linguistics and natural language processing. It contains about 1 million \
words (or "tokens") across a diverse range of texts from 500 sources, categorized into 15 genres, such \
as news, editorial, and fiction, to provide a comprehensive resource for studying the English language. \
This corpus has been instrumental in the development and evaluation of various computational linguistics \
algorithms and tools.
"""
text = text.replace("\n", " ").strip()
print(text)

In [ ]:
nlp = spacy.load('en_core_web_sm')
doc = nlp(text)
print(doc[0], type(doc[0]))

In [ ]:
lemmas = [token.lemma_ for token in doc]
for ori,lemma in zip(doc[:30], lemmas[:30]):
    print(ori, lemma)

**Stemming (词干提取)**

The Porter-Stemmer method

Lemmatization algorithms can be complex. For this reason we sometimes make use of a simpler but cruder method, which mainly consists of chopping off words final affixes. This naive version of morphological analysis is called stemming.

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

text = """\
This was not the map we found in Billy Bones's chest, but \
an accurate copy, complete in all things-names and heights \
and soundings-with the single exception of the red crosses \
and the written notes.\
"""   
doc = nlp(text)

for tok in doc:
    if tok.is_alpha:
        print(tok.text, tok.lemma_)

In [ ]:
# A modern and fast NLP library that includes support for sentence segmentation. 
# spaCy uses a statistical model to predict sentence boundaries, which can be more accurate 
# than rule-based approaches for complex texts.
# Install via conda: conda install conda-forge::spacy
# Install via pip:   pip install -U spacy
# Download data: python -m spacy download en_core_web_sm
import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp("Here is a sentence. Here is another one! And the last one.")
sentences = [sent.text for sent in doc.sents]
for ind, sent in enumerate(sentences):
    print(f"sentence-{ind}: {sent}\n")

In [ ]:
 # You need to install it via: python -m spacy download zh_core_web_sm
from spacy.lang.zh.examples import sentences 
nlp = spacy.load("zh_core_web_sm")
doc = nlp(sentences[0])
text = """\
字节对编码是一种简单的数据压缩形式，这种方法用数据中不存的一个字节表示最常出现的连续字节数据。这样的替换需要重建全部原始数据。字节对编码实例: 假设我们要编码数据 aaabdaaabac, 字节对“aa”出现次数最多，所以我们用数据中没有出现的字节“Z”替换“aa”得到替换表
Z <- aa 数据转变为 ZabdZabac. 在这个数据中，字节对“Za”出现的次数最多，我们用另外一个字节“Y”来替换它（这种情况下由于所有的“Z”都将被替换，所以也可以用“Z”来替换“Za”），得到替换表以及数据
Z <- aa, Y <- Za, YbdYbac. 我们再次替换最常出现的字节对得到：Z <- aa, Y <- Za, X <- Yb. XdXac 由于不再有重复出现的字节对，所以这个数据不能再被进一步压缩。解压的时候，就是按照相反的顺序执行替换过程。
"""
doc = nlp(text)
sentences = [sent.text for sent in doc.sents]
for ind, sent in enumerate(sentences):
    print(f"sentence-{ind}: {sent}\n")

## 4. LLMs tokenization

In [ ]:
!conda install conda-forge::tiktoken -y --quiet

### 4.1 Subword tokenization: BPE

### 4.4 Huggingface tokenizer

PreTrainedTokenizer, PreTrainedTokenizerBase, AutoTokenizer

- https://github.com/huggingface/transformers/blob/main/src/transformers/tokenization_utils.py
- https://github.com/huggingface/transformers/blob/main/src/transformers/tokenization_utils_base.py
- https://github.com/huggingface/transformers/blob/main/src/transformers/tokenization_utils_fast.py
- Check tokenizers at https://github.com/huggingface/transformers/blob/main/setup.py
- If you want to train a tokenizer by yourself, then go to: https://github.com/huggingface/tokenizers
- A fast BPE tokenizer is also at: https://github.com/openai/tiktoken
- An implementation of sentencepiece is at: https://github.com/google/sentencepiece
- There are 3 most common methods for tokenization: https://github.com/huggingface/tokenizers/tree/main/tokenizers/src/models
- - BPE: https://aclanthology.org/P16-1162.pdf
  - Unigram: https://arxiv.org/pdf/1804.10959
  - WordPiece https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/37842.pdf

In [ ]:
from transformers import Qwen2Tokenizer, Qwen2TokenizerFast

## 5. Minimum Edit Distance

In [ ]:
import numpy as np

# define minimum edit distance algorithm via dynamic programming
def minimum_edit_distance(source, target):
    n = len(source)
    m = len(target)
    d_mat = np.zeros((n + 1, m + 1))
    for i in range(1, n + 1):
        d_mat[i, 0] = i
    for j in range(1, m + 1):
        d_mat[0, j] = j
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            sub = 0 if source[i - 1] == target[j - 1] else 2
            del_ = d_mat[i - 1][j] + 1
            ins_ = d_mat[i][j - 1] + 1
            d_mat[i][j] = min(del_, ins_, d_mat[i - 1][j - 1] + sub)
    trace, align_source, align_target = backtrack_alignment(source, target, d_mat)
    return d_mat[n, m], trace, align_source, align_target

def backtrack_alignment(source, target, d_mat):
    align_source, align_target = [], []
    i, j = len(source), len(target)
    back_trace = [[i, j]]

    while (i, j) != (0, 0):
        # boundary cases first (avoid negative indexing)
        if i == 0:
            back_trace.append([i, j - 1])
            align_source = ["*"] + align_source
            align_target = [target[j - 1]] + align_target
            j -= 1
            continue
        if j == 0:
            back_trace.append([i - 1, j])
            align_source = [source[i - 1]] + align_source
            align_target = ["*"] + align_target
            i -= 1
            continue

        sub_cost = 0 if source[i - 1] == target[j - 1] else 2

        # prefer substitution/match when optimal (your tie-break rule)
        if d_mat[i, j] == d_mat[i - 1, j - 1] + sub_cost:
            back_trace.append([i - 1, j - 1])
            align_source = [source[i - 1]] + align_source
            align_target = [target[j - 1]] + align_target
            i, j = i - 1, j - 1

        # deletion
        elif d_mat[i, j] == d_mat[i - 1, j] + 1:
            back_trace.append([i - 1, j])
            align_source = [source[i - 1]] + align_source
            align_target = ["*"] + align_target
            i -= 1

        # insertion
        else:
            back_trace.append([i, j - 1])
            align_source = ["*"] + align_source
            align_target = [target[j - 1]] + align_target
            j -= 1

    return back_trace, align_source, align_target

# test the minimum edit distance
def test_med(source, target):
    med, trace, align_source, align_target = minimum_edit_distance(source, target)
    print(f"input source: {source} and target: {target}")
    print(f"med: {med}")
    print(f"trace: {trace}")
    print(f"aligned source: {align_source}")
    print(f"aligned target: {align_target}")

In [ ]:
test_med(source="INTENTION", target="EXECUTION")

In [ ]:
test_med(source="AGGCTATCACCTGACCTCCAGGCCGATGCCC", target="TAGCTATCACGACCGCGGTCGATTTGCCCGAC")